In [1]:
import os
import sys
%cd ..
print("Current working directory:", os.getcwd())
project_root = os.path.abspath(os.getcwd())
if project_root not in sys.path:
    sys.path.insert(0, project_root)

c:\Users\nicol\OneDrive - University of Bristol\OT2_group_project
Current working directory: c:\Users\nicol\OneDrive - University of Bristol\OT2_group_project


In [3]:

from opentrons.simulate import get_protocol_api
import numpy as np

requirements = {"robotType": "OT-2", "apiLevel": "2.16"}

def run(protocol):

    iteration_count = 7
    wells_per_iteration = 5
    volumes = np.array([[42.06915343, 13.78170516, 19.8832555 , 14.26588591],
 [42.38639251, 12.71361819, 19.40800151, 15.4919878 ],
 [43.02024142, 13.79028736, 18.16751135, 15.02195987],
 [43.1372641 , 14.09678473, 17.9605694 , 14.80538178],
 [39.87355839, 14.85761571, 21.02817419, 14.24065171]])

    #location selected by user when wellplate class created
    well_locs = [8, 5]

    #concentrations used must come in a num of wells x num of liquids size array
    iter_size = volumes.shape[0]
    num_liquids = volumes.shape[1]

    if 1 not in protocol.deck or protocol.deck[1] is None:
        #loading the tips, reservoir and well plate into the program
        tips = protocol.load_labware("opentrons_96_tiprack_1000ul", 1)
        reservoir = protocol.load_labware("nest_12_reservoir_15ml", 2)
        
        plates = {}
        for idx, loc in enumerate(well_locs):
            plates[f"plate_{idx+1}"] = protocol.load_labware("nest_96_wellplate_100ul_pcr_full_skirt", loc)
        
        left_pipette = protocol.load_instrument("p1000_single_gen2", "right", tip_racks=[tips])
    
    else:
        #retrieve existing labware
        tips = protocol.deck[1]
        reservoir = protocol.deck[2]
        plates = {f"plate_{idx+1}": protocol.deck[loc] for idx, loc in enumerate(well_locs)}
        left_pipette = protocol.loaded_instruments["right"]

    start_index = (iteration_count * wells_per_iteration) 
    current_plate_idx = start_index // 96
    plate = plates[f"plate_{current_plate_idx+1}"]  # Get the correct plate
    print(f"plate_{current_plate_idx+1}")

    well_count = iteration_count - current_plate_idx*8

    for liquid in range(num_liquids): 

        left_pipette.pick_up_tip() #one tip for each dye-distribution into all the wells. then a new tip for another color distribution into all the wells. 

        target_wells = []
        for well, volume_set in enumerate(volumes):

            #multiplying by factor of 8: this way we first fill A1 - A12, then B1-B12. instead of A1-H1, then A2-H2.... 
            well_index = well * 8 + well_count

            target_well = plate.wells()[well_index] 
            target_wells.append(target_well)

            
            liquid_volume = volume_set[liquid]
            liquid_source = reservoir[f'A{liquid+1}']

            if liquid != num_liquids - 1:
                left_pipette.transfer(liquid_volume, liquid_source, target_well, new_tip = "never")
            else: 
                left_pipette.transfer(liquid_volume, liquid_source, target_well, new_tip = "never", mix_after=(3, 20 ))

        #bin the tip
        left_pipette.drop_tip()




In [ ]:
protocol = get_protocol_api("2.16") 
run(protocol)

for command in protocol.commands():
    print(command)


### Organized output (1 iteration, population size 5)

plate_1

**(WATER:) Picking up tip from A1 of Opentrons OT-2 96 Tip Rack 1000 µL on slot 1**

- Transferring 42.06915343 from A1 of NEST 12 Well Reservoir 15 mL on slot 2 to H1 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 8
    - Aspirating 42.06915343 uL from A1 of NEST 12 Well Reservoir 15 mL on slot 2 at 274.7 uL/sec
    - Dispensing 42.06915343 uL into H1 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 8 at 274.7 uL/sec
- Transferring 42.38639251 from A1 of NEST 12 Well Reservoir 15 mL on slot 2 to H2 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 8
    - Aspirating 42.38639251 uL from A1 of NEST 12 Well Reservoir 15 mL on slot 2 at 274.7 uL/sec
    - Dispensing 42.38639251 uL into H2 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 8 at 274.7 uL/sec
- Transferring 43.02024142 from A1 of NEST 12 Well Reservoir 15 mL on slot 2 to H3 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 8
    - Aspirating 43.02024142 uL from A1 of NEST 12 Well Reservoir 15 mL on slot 2 at 274.7 uL/sec
    - Dispensing 43.02024142 uL into H3 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 8 at 274.7 uL/sec
- Transferring 43.1372641 from A1 of NEST 12 Well Reservoir 15 mL on slot 2 to H4 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 8
    - Aspirating 43.1372641 uL from A1 of NEST 12 Well Reservoir 15 mL on slot 2 at 274.7 uL/sec
    - Dispensing 43.1372641 uL into H4 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 8 at 274.7 uL/sec
- Transferring 39.87355839 from A1 of NEST 12 Well Reservoir 15 mL on slot 2 to H5 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 8
    - Aspirating 39.87355839 uL from A1 of NEST 12 Well Reservoir 15 mL on slot 2 at 274.7 uL/sec
    - Dispensing 39.87355839 uL into H5 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 8 at 274.7 uL/sec

**Dropping tip into Trash Bin on slot 12**

**(BLUE:) Picking up tip from B1 of Opentrons OT-2 96 Tip Rack 1000 µL on slot 1**

- Transferring 13.78170516 from A2 of NEST 12 Well Reservoir 15 mL on slot 2 to H1 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 8
    - Aspirating 13.78170516 uL from A2 of NEST 12 Well Reservoir 15 mL on slot 2 at 274.7 uL/sec
    - Dispensing 13.78170516 uL into H1 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 8 at 274.7 uL/sec
- Transferring 12.71361819 from A2 of NEST 12 Well Reservoir 15 mL on slot 2 to H2 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 8
    - Aspirating 12.71361819 uL from A2 of NEST 12 Well Reservoir 15 mL on slot 2 at 274.7 uL/sec
    - Dispensing 12.71361819 uL into H2 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 8 at 274.7 uL/sec
- Transferring 13.79028736 from A2 of NEST 12 Well Reservoir 15 mL on slot 2 to H3 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 8
    - Aspirating 13.79028736 uL from A2 of NEST 12 Well Reservoir 15 mL on slot 2 at 274.7 uL/sec
    - Dispensing 13.79028736 uL into H3 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 8 at 274.7 uL/sec
- Transferring 14.09678473 from A2 of NEST 12 Well Reservoir 15 mL on slot 2 to H4 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 8
    - Aspirating 14.09678473 uL from A2 of NEST 12 Well Reservoir 15 mL on slot 2 at 274.7 uL/sec
    - Dispensing 14.09678473 uL into H4 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 8 at 274.7 uL/sec
- Transferring 14.85761571 from A2 of NEST 12 Well Reservoir 15 mL on slot 2 to H5 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 8
    - Aspirating 14.85761571 uL from A2 of NEST 12 Well Reservoir 15 mL on slot 2 at 274.7 uL/sec
    - Dispensing 14.85761571 uL into H5 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 8 at 274.7 uL/sec

**Dropping tip into Trash Bin on slot 12**

**(YELLOW:) Picking up tip from C1 of Opentrons OT-2 96 Tip Rack 1000 µL on slot 1**

- Transferring 19.8832555 from A3 of NEST 12 Well Reservoir 15 mL on slot 2 to H1 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 8
    - Aspirating 19.8832555 uL from A3 of NEST 12 Well Reservoir 15 mL on slot 2 at 274.7 uL/sec
    - Dispensing 19.8832555 uL into H1 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 8 at 274.7 uL/sec
- Transferring 19.40800151 from A3 of NEST 12 Well Reservoir 15 mL on slot 2 to H2 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 8
    - Aspirating 19.40800151 uL from A3 of NEST 12 Well Reservoir 15 mL on slot 2 at 274.7 uL/sec
    - Dispensing 19.40800151 uL into H2 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 8 at 274.7 uL/sec
- Transferring 18.16751135 from A3 of NEST 12 Well Reservoir 15 mL on slot 2 to H3 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 8
    - Aspirating 18.16751135 uL from A3 of NEST 12 Well Reservoir 15 mL on slot 2 at 274.7 uL/sec
    - Dispensing 18.16751135 uL into H3 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 8 at 274.7 uL/sec
- Transferring 17.9605694 from A3 of NEST 12 Well Reservoir 15 mL on slot 2 to H4 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 8
    - Aspirating 17.9605694 uL from A3 of NEST 12 Well Reservoir 15 mL on slot 2 at 274.7 uL/sec
    - Dispensing 17.9605694 uL into H4 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 8 at 274.7 uL/sec
- Transferring 21.02817419 from A3 of NEST 12 Well Reservoir 15 mL on slot 2 to H5 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 8
    - Aspirating 21.02817419 uL from A3 of NEST 12 Well Reservoir 15 mL on slot 2 at 274.7 uL/sec
    - Dispensing 21.02817419 uL into H5 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 8 at 274.7 uL/sec

**Dropping tip into Trash Bin on slot 12**

**(RED + mixing as it's the last liquid:) Picking up tip from D1 of Opentrons OT-2 96 Tip Rack 1000 µL on slot 1**

- Transferring 14.26588591 from A4 of NEST 12 Well Reservoir 15 mL on slot 2 to H1 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 8
    - Aspirating 14.26588591 uL from A4 of NEST 12 Well Reservoir 15 mL on slot 2 at 274.7 uL/sec
    - Dispensing 14.26588591 uL into H1 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 8 at 274.7 uL/sec
- Mixing 3 times with a volume of 20.0 ul
    - Aspirating 20.0 uL from H1 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 8 at 274.7 uL/sec
    - Dispensing 20.0 uL into H1 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 8 at 274.7 uL/sec
    - Aspirating 20.0 uL from H1 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 8 at 274.7 uL/sec
    - Dispensing 20.0 uL into H1 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 8 at 274.7 uL/sec
    - Aspirating 20.0 uL from H1 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 8 at 274.7 uL/sec
    - Dispensing 20.0 uL into H1 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 8 at 274.7 uL/sec
- Transferring 15.4919878 from A4 of NEST 12 Well Reservoir 15 mL on slot 2 to H2 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 8
    - Aspirating 15.4919878 uL from A4 of NEST 12 Well Reservoir 15 mL on slot 2 at 274.7 uL/sec
    - Dispensing 15.4919878 uL into H2 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 8 at 274.7 uL/sec
- Mixing 3 times with a volume of 20.0 ul
    - Aspirating 20.0 uL from H2 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 8 at 274.7 uL/sec
    - Dispensing 20.0 uL into H2 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 8 at 274.7 uL/sec
    - Aspirating 20.0 uL from H2 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 8 at 274.7 uL/sec
    - Dispensing 20.0 uL into H2 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 8 at 274.7 uL/sec
    - Aspirating 20.0 uL from H2 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 8 at 274.7 uL/sec
    - Dispensing 20.0 uL into H2 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 8 at 274.7 uL/sec
- Transferring 15.02195987 from A4 of NEST 12 Well Reservoir 15 mL on slot 2 to H3 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 8
    - Aspirating 15.02195987 uL from A4 of NEST 12 Well Reservoir 15 mL on slot 2 at 274.7 uL/sec
    - Dispensing 15.02195987 uL into H3 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 8 at 274.7 uL/sec
- Mixing 3 times with a volume of 20.0 ul
    - Aspirating 20.0 uL from H3 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 8 at 274.7 uL/sec
    - Dispensing 20.0 uL into H3 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 8 at 274.7 uL/sec
    - Aspirating 20.0 uL from H3 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 8 at 274.7 uL/sec
    - Dispensing 20.0 uL into H3 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 8 at 274.7 uL/sec
    - Aspirating 20.0 uL from H3 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 8 at 274.7 uL/sec
    - Dispensing 20.0 uL into H3 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 8 at 274.7 uL/sec
- Transferring 14.80538178 from A4 of NEST 12 Well Reservoir 15 mL on slot 2 to H4 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 8
    - Aspirating 14.80538178 uL from A4 of NEST 12 Well Reservoir 15 mL on slot 2 at 274.7 uL/sec
    - Dispensing 14.80538178 uL into H4 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 8 at 274.7 uL/sec
- Mixing 3 times with a volume of 20.0 ul
    - Aspirating 20.0 uL from H4 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 8 at 274.7 uL/sec
    - Dispensing 20.0 uL into H4 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 8 at 274.7 uL/sec
    - Aspirating 20.0 uL from H4 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 8 at 274.7 uL/sec
    - Dispensing 20.0 uL into H4 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 8 at 274.7 uL/sec
    - Aspirating 20.0 uL from H4 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 8 at 274.7 uL/sec
    - Dispensing 20.0 uL into H4 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 8 at 274.7 uL/sec
- Transferring 14.24065171 from A4 of NEST 12 Well Reservoir 15 mL on slot 2 to H5 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 8
    - Aspirating 14.24065171 uL from A4 of NEST 12 Well Reservoir 15 mL on slot 2 at 274.7 uL/sec
    - Dispensing 14.24065171 uL into H5 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 8 at 274.7 uL/sec
- Mixing 3 times with a volume of 20.0 ul
    - Aspirating 20.0 uL from H5 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 8 at 274.7 uL/sec
    - Dispensing 20.0 uL into H5 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 8 at 274.7 uL/sec
    - Aspirating 20.0 uL from H5 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 8 at 274.7 uL/sec
    - Dispensing 20.0 uL into H5 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 8 at 274.7 uL/sec
    - Aspirating 20.0 uL from H5 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 8 at 274.7 uL/sec
    - Dispensing 20.0 uL into H5 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 8 at 274.7 uL/sec

**Dropping tip into Trash Bin on slot 12**